In [1]:
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm
from specbuffer import SpecReplayBuffer
from buffer import ReplayBuffer
from PPOee import PPO
import gym
from CustomEnv import CustomEnv
from Student_Agent import StudentAgent
from TeacherAgent import TeacherAgent
from ExpertTrajectory import ExpertTrajectory

import contextlib
import network_sim
from stable_baselines3 import PPO as sb3ppo
import pickle

cuda:0


2024-04-01 10:47:15.656837: I tensorflow/stream_executor/platform/default/dso_loader.cc:50] Successfully opened dynamic library libcudart.so.12


In [2]:
env=CustomEnv('PccNs-v0',1200)

History length: 10
Features: ['sent latency inflation', 'latency ratio', 'send ratio']
Getting min obs for ['sent latency inflation', 'latency ratio', 'send ratio']


In [3]:
et=ExpertTrajectory(1600)

In [4]:
et.load_expert_model('verygood.zip')
et.generate_trajectory(env,40)

400it [00:01, 386.39it/s]                     


In [5]:
class Student(StudentAgent):
    def __init__(self, state_dim, action_dim, env):
        super().__init__(state_dim, action_dim, env)
        
    def generate_trajectory(self, step: int):
        pb=tqdm(range(step))
        for _ in pb:
            s=self.env.reset()
            d=False
            while not d:
                a,action,l,v=self.model.select_action(s)
                s_,r,d,_=self.env.step(a)
                self.model.buffer.store(s[-3:],a,l,s_,r,v,d)
                s=s_
            pb.update()

In [6]:
sa=Student(30,1,env)

In [7]:
sa.generate_trajectory(10)

100%|██████████| 10/10 [00:03<00:00,  2.76it/s]


In [8]:
sa.train(10,8)

  0%|          | 0/10 [00:00<?, ?it/s]/home/data/envs/IL/lib/python3.8/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
100%|██████████| 10/10 [00:01<00:00,  8.12it/s]


In [9]:
class Teacher(TeacherAgent):
    def __init__(self, state_dim, action_dim, env:CustomEnv, expert_trajectory):
        super().__init__(state_dim, action_dim, env, expert_trajectory)
        
    def ComputeReward(self):
        pb=tqdm(range(min(self.trajectory_buffer.index,self.trajectory_buffer.buffer_size)))
        for i in pb:
            length=len(self.trajectory_buffer.state[0])
            s=torch.FloatTensor(np.array(self.trajectory_buffer.get_state_padding(self.trajectory_buffer.state[int(i/length)],i%length,10,True)[1])).view(-1)
            sa_pair=torch.cat((s,self.trajectory_buffer.action[i]),-1)
            reward,_,l,v=self.model.select_action(sa_pair)
            self.replay_buffer.store(
                sa_pair,
                reward,
                l,
                self.trajectory_buffer.next_state[i],
                self.discriminator.model(sa_pair).detach().cpu().numpy(),
                v,
                self.trajectory_buffer.done[i]
            )
            self.trajectory_buffer.reward[i]=reward
            pb.update()
        self.discriminator.collect_expert()

In [10]:
ta=Teacher(30,1,env,et)

In [11]:
ta.ComputeReward()
#BUG 这里有问题

100%|██████████| 1200/1200 [00:01<00:00, 688.22it/s]


In [12]:
sa.train(10,8)

  0%|          | 0/10 [00:00<?, ?it/s]/home/tools/DR/MyProject/RILE/PPOee.py:191: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  rewards = torch.tensor(rewards, dtype=torch.float32).to(device)
100%|██████████| 10/10 [00:01<00:00,  8.30it/s]


In [13]:
ta.model.buffer

In [14]:
ta.train(10,10,10)

  0%|          | 0/10 [00:00<?, ?it/s]

Computing reward...


100%|██████████| 1200/1200 [00:01<00:00, 686.58it/s]
/home/data/envs/IL/lib/python3.8/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1024])) that is different to the input size (torch.Size([1024, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Training Discriminator...
Training PPO...


 10%|█         | 1/10 [00:03<00:31,  3.51s/it]

...

13it [00:30,  2.51s/it]

Computing reward...


100%|██████████| 1200/1200 [00:00<00:00, 1315.95it/s]


Training Discriminator...
Training PPO...


100%|██████████| 10/10 [00:33<00:00,  3.31s/it]


In [15]:
import importlib
module=importlib.import_module('Discriminator','Discriminator.py')


In [16]:
d=module.Discriminator(31,1,8,sa.replay_buffer,et)

In [17]:
d.collect_expert()

In [18]:
d.collect_dist(8)

In [19]:
d.update(100,False)